In [2]:
import pandas as pd 
import numpy as np

In [3]:
df = pd.read_csv("child_data.csv")
kits = pd.read_csv("play_kit_data.csv")

In [4]:
df.head()

,Age,Developmental_Stage,Cognitive_Ability,Interest,Play_Style,Special_Needs
0,3.8,Toddler,Shape Recognition,Puzzles,Group,NaN
1,3.9,Toddler,Basic Problem Solving,Drawing,Solo,NaN
2,1.8,Infant,Sensory Exploration,Music,Group,NaN
3,11.7,Middle Childhood,Analytical Thinking,Robotics,Interactive Digital,NaN
4,0.7,Infant,Grasping,Music,Interactive Digital,NaN


In [5]:
kits.head()

,Kit_Name,Recommended_Age,Stage,Cognitive_Skill_Targeted,Interest
0,Soft Sensory Mat,0.5-2,Infant,Sensory Exploration,Soft Toys
1,Colorful Stacking Blocks,2-5,Toddler,Basic Problem Solving,Building Blocks
2,Shape Sorting Cube,2-5,Toddler,Shape Recognition,Puzzles
3,STEM Learning Kit,5-8,Early Childhood,Logical Thinking,STEM Kits
4,Memory Card Game,5-8,Early Childhood,Memory,Board Games


In [11]:

print(df["Developmental_Stage"].unique())
print(df["Cognitive_Ability"].unique())
print(df["Interest"].unique())
print(df["Play_Style"].unique())
print(df["Special_Needs"].unique())




['Toddler' 'Infant' 'Middle Childhood' 'Early Childhood' 'Adolescence']
['Shape Recognition' 'Basic Problem Solving' 'Sensory Exploration'
 'Analytical Thinking' 'Grasping' 'Logical Thinking' 'Abstract Reasoning'
 'Creativity' 'Critical Thinking' 'Memory']
['Puzzles' 'Drawing' 'Music' 'Robotics' 'Storytelling' 'Engineering'
 'Soft Toys' 'Rattles' 'Strategy Games' 'AI & ML Kits' 'Advanced Puzzles'
 'STEM Kits' 'Coding' 'Building Blocks' 'Board Games']
['Group' 'Solo' 'Interactive Digital']
[nan 'Dyslexia' 'Autism' 'ADHD']


In [9]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity

def load_data():
    """ Load child and play kit data """
    child_df = pd.read_csv("child_data.csv")
    play_kit_df = pd.read_csv("play_kit_data.csv")
    return child_df, play_kit_df

def preprocess_data(child_df, play_kit_df):
    """ Encode categorical features and prepare data """
    label_encoders = {}
    for col in ["Developmental_Stage", "Cognitive_Ability", "Interest", "Play_Style", "Special_Needs"]:
        le = LabelEncoder()
        child_df[col] = le.fit_transform(child_df[col])
        label_encoders[col] = le
    
    for col in ["Stage", "Cognitive_Skill_Targeted", "Interest"]:
        le = LabelEncoder()
        play_kit_df[col] = le.fit_transform(play_kit_df[col])
        label_encoders[col] = le
    
    play_kit_df["Min_Age"] = play_kit_df["Recommended_Age"].apply(lambda x: float(x.split("-")[0]))
    play_kit_df["Max_Age"] = play_kit_df["Recommended_Age"].apply(lambda x: float(x.split("-")[1]))
    
    child_df["Normalized_Age"] = (child_df["Age"] - child_df["Age"].min()) / (child_df["Age"].max() - child_df["Age"].min())
    play_kit_df["Normalized_Age"] = (play_kit_df["Min_Age"] + play_kit_df["Max_Age"]) / 2
    play_kit_df["Normalized_Age"] = (play_kit_df["Normalized_Age"] - child_df["Age"].min()) / (child_df["Age"].max() - child_df["Age"].min())
    
    return child_df, play_kit_df, label_encoders

def train_and_save_model():
    """ Train AI-powered play kit recommendation system and save it """
    child_df, play_kit_df = load_data()
    child_df, play_kit_df, label_encoders = preprocess_data(child_df, play_kit_df)
    
    child_features = child_df[["Normalized_Age", "Developmental_Stage", "Cognitive_Ability", "Interest", "Play_Style", "Special_Needs"]]
    play_kit_features = play_kit_df[["Normalized_Age", "Stage", "Cognitive_Skill_Targeted", "Interest", "Min_Age", "Max_Age"]]
    
    similarity_matrix = cosine_similarity(child_features, play_kit_features)
    joblib.dump((similarity_matrix, label_encoders, play_kit_df, child_df), "playkit_recommender.pkl")
    print("✅ Model trained and saved!")

def load_model():
    """ Load the saved model for inference """
    return joblib.load("playkit_recommender.pkl")

def recommend_play_kits(new_child):
    """ Recommend play kits for a new child using the saved model """
    similarity_matrix, label_encoders, play_kit_df, child_df = load_model()
    
    new_child_df = pd.DataFrame([new_child])
    for col in ["Developmental_Stage", "Cognitive_Ability", "Interest", "Play_Style", "Special_Needs"]:
        new_child_df[col] = label_encoders[col].transform([new_child[col]])[0]
    
    new_child_df["Normalized_Age"] = (new_child_df["Age"] - child_df["Age"].min()) / (child_df["Age"].max() - child_df["Age"].min())
    new_child_features = new_child_df[["Normalized_Age", "Developmental_Stage", "Cognitive_Ability", "Interest", "Play_Style", "Special_Needs"]]
    
    play_kit_features = play_kit_df[["Normalized_Age", "Stage", "Cognitive_Skill_Targeted", "Interest", "Min_Age", "Max_Age"]]
    similarity_scores = cosine_similarity(new_child_features, play_kit_features)
    top_kits = np.argsort(similarity_scores, axis=1)[:, -3:][0]
    recommended_play_kits = play_kit_df.iloc[top_kits]["Kit_Name"].values.tolist()
    
    return recommended_play_kits

# Train and Save Model
train_and_save_model()

# Example Inference
new_child = {
    "Age": 6,
    "Developmental_Stage": "Early Childhood",
    "Cognitive_Ability": "Logical Thinking",
    "Interest": "STEM Kits",
    "Play_Style": "Solo",
    "Special_Needs": "None"
}

recommendations = recommend_play_kits(new_child)
print("Recommended Play Kits:", recommendations)


✅ AI Model Trained & Recommendations Saved!
Recommended Play Kits: ['STEM Learning Kit', 'Shape Sorting Cube', 'Soft Sensory Mat']
